<a href="https://colab.research.google.com/github/songqsh/18ma573pub/blob/bra1/src/bsm_calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- git clone repo to colab

In [0]:
'''======
run once, then comment it off, and run again
========'''
#!git clone https://github.com/songqsh/18ma573pub.git 
pass

- Change working dir to "reponame/src", also needs run once, and comment it out from the second run.

In [0]:
#cd 18ma573pub/src/

- here we import our own python modules and common modules

In [0]:
import european_options_class as eoc
import sde_class as sde
import numpy as np

- Read data from .dat file. It has call option prices of
  - maturities 2 months and 5 months;
  - strikes of 97, 99, 101, 103

In [44]:
'''======
Read data
========='''
#Read four-column data
#columns are otype, maturity, strike, option_price
np_option_data1 = np.loadtxt('optiondata2.dat', comments='#', delimiter=',')

print('>>>>>>otype, maturity, strike, option_price')
print(np_option_data1)

>>>>>>otype, maturity, strike, option_price
[[  1.           0.16666667  97.           5.32705461]
 [  1.           0.16666667  99.           3.86224255]
 [  1.           0.16666667 101.           2.7204371 ]
 [  1.           0.16666667 103.           2.1202793 ]
 [  1.           0.41666667  97.           7.23756307]
 [  1.           0.41666667  99.           5.95053461]
 [  1.           0.41666667 101.           5.2640122 ]
 [  1.           0.41666667 103.           4.97493422]]


- We also need to give spot price and interest rate before the computation.

In [0]:
'''==============
below are from the market data for underlying process
================='''
gbm1 = sde.Gbm(
    init_state = 100, #market data
    drift_ratio = .05, #market data
    vol_ratio = .1 #initial guess
)

- construct an option class including market price as its member. Essentially the same as EuropeanOption class with slight modifications.

In [0]:
'''=========
option class init
=========='''
class VanillaOption:
    def __init__(
        self,
        otype = 1, # 1: 'call'
                  # -1: 'put'
        strike = 110.,
        maturity = 1.,
        market_price = 10.):
      self.otype = otype
      self.strike = strike
      self.maturity = maturity
      self.market_price = market_price
      
        
    def payoff(self, s): #s: excercise price
      otype = self.otype
      k = self.strike
      maturity = self.maturity
      return np.max([0, (s - k)*otype])

- creat option_list from the data

In [0]:
num_row = np_option_data1.shape[0]
option_list = []

for i in range(num_row):
  option1 = VanillaOption(
      otype = np_option_data1[i,0],
      strike = np_option_data1[i,2],
      maturity = np_option_data1[i,1],
      market_price = np_option_data1[i,3]
  )
      
  option_list.append(option1) 
  

- Define calibration method

In [0]:
'''==========
define a method to seek for an calibrated volatility
============'''

def error_function(vol, gbm, option_list):
  gbm.vol_ratio = vol
  err1 = 0
  for i in len(option_list):
    err1 = err1 + np.abs(option_list[i].market_price 
                         - gbm.bsm_price(option_list[i]))
    
  return err1

In [0]:
'''==========
define a method to seek for an calibrated volatility
============'''

def error_function(vol, gbm, option, market_option_price):
  gbm.vol_ratio = vol
  return np.abs(market_option_price - gbm.bsm_price(option))

def implied_volatility(gbm, option, market_option_price):
  init_vol = .1 #initial guess
  return so.fmin(error_function, init_vol, 
                 args = (gbm, option, market_option_price), disp = 0)[0]

In [0]:
class Dog:
  name = 'tom'
  def __init__(self, age=5):
    self.age = age
  def out(self):
    print(self.age)

In [12]:
dog1 = Dog(3)
dog1.out()

3


In [0]:
dog_list = [Dog(i) for i in range(5)]

In [0]:
def f(dog_list):
  for i in range(len(dog_list)):
    dog_list[i].out()

In [23]:
f(dog_list)

0
1
2
3
4
